In [1]:
import keras
import librosa

DATA_PATH = 'E:\\Datasets\\Speech\\train\\audio\\'

Using TensorFlow backend.


# Process audio files

In [ ]:
from tqdm import tqdm

def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '\\' + wavfile for wavfile in os.listdir(path + '\\' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save('data\\speech\\'+label + '.npy', mfcc_vectors)
        print('saved data : ', label)
        
save_data_to_array()

In [2]:
import numpy as np
import os
from keras.utils import to_categorical

def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [3]:
get_labels()

(['bed',
  'bird',
  'cat',
  'dog',
  'down',
  'eight',
  'five',
  'four',
  'go',
  'happy',
  'house',
  'left',
  'marvin',
  'nine',
  'no',
  'off',
  'on',
  'one',
  'right',
  'seven',
  'sheila',
  'six',
  'stop',
  'three',
  'tree',
  'two',
  'up',
  'wow',
  'yes',
  'zero'],
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0., 

In [4]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.9, random_state=42):
    # get available class labels
    labels, indices, _ = get_labels()
    
    # Getting first arrays
    X = np.load('data\\speech\\'+labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load('data\\speech\\'+label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)


In [5]:
train_x, test_x, train_y, test_y = get_train_test()

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(58248, 20, 11)
(6473, 20, 11)
(58248,)
(6473,)


In [6]:
x, y, z = train_x.shape
train_x = train_x.reshape(x, y, z, 1)
test_x = test_x.reshape(test_x.shape[0], y, z, 1)

train_y_categorical = to_categorical(train_y)
test_y_categorical = to_categorical(test_y)

print(train_x.shape)
print(test_x.shape)
print(train_y_categorical.shape)
print(test_y_categorical.shape)

(58248, 20, 11, 1)
(6473, 20, 11, 1)
(58248, 30)
(6473, 30)


# Build the netowrk

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(20, 11, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 19, 10, 32)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 5, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1440)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               184448    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3870      
Total para

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['categorical_accuracy'])
model.fit(train_x, train_y_categorical, batch_size=100, epochs=200, validation_data=(test_x, test_y_categorical))

Train on 58248 samples, validate on 6473 samples
Epoch 1/200
58248/58248 [==============================] - 4s 77us/step - loss: 6.4840 - categorical_accuracy: 0.0497 - val_loss: 3.2044 - val_categorical_accuracy: 0.0776
Epoch 2/200
58248/58248 [==============================] - 3s 43us/step - loss: 3.0790 - categorical_accuracy: 0.1053 - val_loss: 2.8650 - val_categorical_accuracy: 0.1628
Epoch 3/200
58248/58248 [==============================] - 3s 44us/step - loss: 2.9378 - categorical_accuracy: 0.1367 - val_loss: 2.6740 - val_categorical_accuracy: 0.2107
Epoch 4/200
58248/58248 [==============================] - 3s 44us/step - loss: 2.8154 - categorical_accuracy: 0.1687 - val_loss: 2.4922 - val_categorical_accuracy: 0.2685
Epoch 5/200
58248/58248 [==============================] - 3s 45us/step - loss: 2.6621 - categorical_accuracy: 0.2052 - val_loss: 2.3128 - val_categorical_accuracy: 0.3352
Epoch 6/200
58248/58248 [==============================] - 3s 46us/step - loss: 2.5082 - ca

58248/58248 [==============================] - 3s 44us/step - loss: 1.4237 - categorical_accuracy: 0.5691 - val_loss: 1.0619 - val_categorical_accuracy: 0.6926
Epoch 49/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.4201 - categorical_accuracy: 0.5706 - val_loss: 1.1595 - val_categorical_accuracy: 0.6651
Epoch 50/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.4076 - categorical_accuracy: 0.5779 - val_loss: 1.0453 - val_categorical_accuracy: 0.6958
Epoch 51/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.3958 - categorical_accuracy: 0.5784 - val_loss: 1.0678 - val_categorical_accuracy: 0.6907
Epoch 52/200
58248/58248 [==============================] - 2s 43us/step - loss: 1.4033 - categorical_accuracy: 0.5758 - val_loss: 1.0672 - val_categorical_accuracy: 0.6946
Epoch 53/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.3961 - categorical_accuracy: 0.5768 - val_loss: 1.0639 - val_cate

58248/58248 [==============================] - 2s 42us/step - loss: 1.2380 - categorical_accuracy: 0.6252 - val_loss: 0.9282 - val_categorical_accuracy: 0.7273
Epoch 96/200
58248/58248 [==============================] - 2s 42us/step - loss: 1.2423 - categorical_accuracy: 0.6259 - val_loss: 0.9394 - val_categorical_accuracy: 0.7232
Epoch 97/200
58248/58248 [==============================] - 2s 42us/step - loss: 1.2307 - categorical_accuracy: 0.6283 - val_loss: 0.9266 - val_categorical_accuracy: 0.7326
Epoch 98/200
58248/58248 [==============================] - 3s 43us/step - loss: 1.2346 - categorical_accuracy: 0.6248 - val_loss: 0.8925 - val_categorical_accuracy: 0.7397
Epoch 99/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.2421 - categorical_accuracy: 0.6218 - val_loss: 0.9454 - val_categorical_accuracy: 0.7176
Epoch 100/200
58248/58248 [==============================] - 3s 45us/step - loss: 1.2383 - categorical_accuracy: 0.6249 - val_loss: 0.9401 - val_cat

58248/58248 [==============================] - 2s 43us/step - loss: 1.1902 - categorical_accuracy: 0.6417 - val_loss: 0.9001 - val_categorical_accuracy: 0.7372
Epoch 143/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.1887 - categorical_accuracy: 0.6375 - val_loss: 0.9523 - val_categorical_accuracy: 0.7218
Epoch 144/200
58248/58248 [==============================] - 3s 46us/step - loss: 1.1815 - categorical_accuracy: 0.6422 - val_loss: 0.9325 - val_categorical_accuracy: 0.7338
Epoch 145/200
58248/58248 [==============================] - 3s 47us/step - loss: 1.1864 - categorical_accuracy: 0.6412 - val_loss: 0.9166 - val_categorical_accuracy: 0.7292
Epoch 146/200
58248/58248 [==============================] - 3s 47us/step - loss: 1.1832 - categorical_accuracy: 0.6420 - val_loss: 0.9196 - val_categorical_accuracy: 0.7253
Epoch 147/200
58248/58248 [==============================] - 3s 47us/step - loss: 1.1858 - categorical_accuracy: 0.6409 - val_loss: 0.8911 - val

Epoch 189/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.1664 - categorical_accuracy: 0.6455 - val_loss: 0.9030 - val_categorical_accuracy: 0.7403
Epoch 190/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.1696 - categorical_accuracy: 0.6437 - val_loss: 0.8803 - val_categorical_accuracy: 0.7403
Epoch 191/200
58248/58248 [==============================] - 3s 44us/step - loss: 1.1692 - categorical_accuracy: 0.6474 - val_loss: 0.9143 - val_categorical_accuracy: 0.7351
Epoch 192/200
58248/58248 [==============================] - 3s 45us/step - loss: 1.1765 - categorical_accuracy: 0.6453 - val_loss: 0.9101 - val_categorical_accuracy: 0.7318
Epoch 193/200
58248/58248 [==============================] - 3s 45us/step - loss: 1.1597 - categorical_accuracy: 0.6480 - val_loss: 0.9112 - val_categorical_accuracy: 0.7343
Epoch 194/200
58248/58248 [==============================] - 3s 45us/step - loss: 1.1622 - categorical_accuracy: 0.6491 - val_loss

# LSTM 

In [16]:
train_x, test_x, train_y, test_y = get_train_test()

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(58248, 20, 11)
(6473, 20, 11)
(58248,)
(6473,)


In [17]:
from keras import layers

model = Sequential()

model.add(layers.LSTM(128, activation='relu', input_shape=(None, 11)))
model.add(layers.Dense(30, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               71680     
_________________________________________________________________
dense_4 (Dense)              (None, 30)                3870      
Total params: 75,550
Trainable params: 75,550
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['categorical_accuracy'])
model.fit(train_x, train_y_categorical, batch_size=100, epochs=200, validation_data=(test_x, test_y_categorical))

Train on 58248 samples, validate on 6473 samples
Epoch 1/200
58248/58248 [==============================] - 13s 226us/step - loss: 3.9196 - categorical_accuracy: 0.0458 - val_loss: 3.3594 - val_categorical_accuracy: 0.0621
Epoch 2/200
58248/58248 [==============================] - 13s 219us/step - loss: 3.2256 - categorical_accuracy: 0.0869 - val_loss: 3.0730 - val_categorical_accuracy: 0.1089
Epoch 3/200
58248/58248 [==============================] - 13s 222us/step - loss: 2.8487 - categorical_accuracy: 0.1585 - val_loss: 2.6785 - val_categorical_accuracy: 0.1931
Epoch 4/200
58248/58248 [==============================] - 13s 215us/step - loss: 2.5272 - categorical_accuracy: 0.2381 - val_loss: 2.3977 - val_categorical_accuracy: 0.2755
Epoch 5/200
58248/58248 [==============================] - 13s 219us/step - loss: 2.2058 - categorical_accuracy: 0.3276 - val_loss: 2.1044 - val_categorical_accuracy: 0.3521
Epoch 6/200
58248/58248 [==============================] - 13s 231us/step - loss: